In [1]:
import warnings
warnings.simplefilter(action='ignore', category=FutureWarning)
from IPython.display import display, Markdown

import pandas as pd
import numpy as np

import json

np.__version__

'1.23.4'

In [2]:
gender_df = pd.read_csv('./data/11-OUTPUT-with-gender-from-gpt4.csv')
date_as_decimal = (gender_df.chart_debut_date_normalized_0_to_1 / 100) * (2023 - 1959) + 1959
date_as_decimal
gender_df['date_as_decimal'] = date_as_decimal
gender_df

,performer,song,generic_genre,lyric_line,chart_debut,x,y,chart_debut_date_normalized_0_to_1,genre_position_band,near_neighbors_count,gender,date_as_decimal
0,John Lennon,#9 Dream,ROCK,So long ago,1974-12-21,62.357605,41.475302,25.450,10.0,14,m,1975.28800
1,John Lennon,#9 Dream,ROCK,"Was it in a dream, was it just a dream?",1974-12-21,55.684884,27.032071,25.450,10.0,14,m,1975.28800
2,John Lennon,#9 Dream,ROCK,"I know, yes I know",1974-12-21,69.964586,46.613546,25.450,10.0,0,m,1975.28800
3,John Lennon,#9 Dream,ROCK,"Seemed so very real, it seemed so real to me",1974-12-21,60.181471,69.527274,25.450,10.0,14,m,1975.28800
4,John Lennon,#9 Dream,ROCK,Took a walk down the street,1974-12-21,49.938720,41.965371,25.450,10.0,0,m,1975.28800
...,...,...,...,...,...,...,...,...,...,...,...,...
167355,J. Cole & Lil Baby,pride.is.the.devil,HIP_HOP,"I'm addicted to promethazine, it's crazy, yeah...",2021-05-29,41.610927,50.449968,97.562,20.0,0,m,2021.43968
167356,J. Cole & Lil Baby,pride.is.the.devil,HIP_HOP,"All this money coming in, it drive me crazy no...",2021-05-29,45.775471,52.982471,97.562,20.0,0,m,2021.43968
167357,J. Cole & Lil Baby,pride.is.the.devil,HIP_HOP,I'll be crazy if I blow it,2021-05-29,45.995316,47.344963,97.562,20.0,5,m,2021.43968
167358,J. Cole & Lil Baby,pride.is.the.devil,HIP_HOP,And it left so many R.I.P.,2021-05-29,57.715991,46.589529,97.562,20.0,0,m,2021.43968


# Setup GPT3 call

In [3]:
import os
import openai
# openai.organization = "personal"
openai.api_key = os.getenv("OPENAI_API_KEY")
[model.id for model in openai.Model.list()['data']]

['text-davinci-001',
 'text-search-curie-query-001',
 'davinci',
 'gpt-4-0613',
 'text-babbage-001',
 'curie-instruct-beta',
 'text-davinci-003',
 'gpt-3.5-turbo-16k-0613',
 'davinci-similarity',
 'code-davinci-edit-001',
 'text-similarity-curie-001',
 'gpt-3.5-turbo-16k',
 'text-embedding-ada-002',
 'ada-code-search-text',
 'text-search-ada-query-001',
 'gpt-4-0314',
 'babbage-search-query',
 'ada-similarity',
 'gpt-3.5-turbo',
 'text-search-ada-doc-001',
 'text-search-babbage-query-001',
 'code-search-ada-code-001',
 'curie-search-document',
 'text-search-davinci-query-001',
 'text-search-curie-doc-001',
 'gpt-3.5-turbo-0301',
 'babbage-search-document',
 'babbage-code-search-text',
 'davinci-instruct-beta',
 'davinci-search-query',
 'text-similarity-babbage-001',
 'text-davinci-002',
 'code-search-babbage-text-001',
 'babbage',
 'text-search-davinci-doc-001',
 'code-search-ada-text-001',
 'ada-search-query',
 'text-similarity-ada-001',
 'whisper-1',
 'gpt-4',
 'ada-code-search-code'

In [4]:
LOVE_SONG_LABELING_PROMPT = """
I will to give you a JSON list where each item is an array that contains an array where the first item is the performer and the second is the name of one of their songs from the Billboard Top 10 from 1958-2021.

Here's an example input, with 2 columns ('performer', 'song'):

```
[
    ['twenty one pilots', 'Heathens'],
    ['Little Mix and Jason Derulo'],
    ['will.i.am & Britney Spears', 'Scream & Shout'],
    ["21 Savage & Metro Boomin", "Runnin"],
    ["6ix9ine Featuring Nicki Minaj & Murda Beatz", "FEFE"],
    ['2Pac', "Dear Mama/Old School"],
    ["'N Sync", 'This I Promise You'],
    ["Katy Perry", "Hot N Cold"],
    ["112", "Peaches & Cream"],
    ["10cc", "I'm Not In Love"],
    ["Olivia Rodrigo", "good 4 u"],
]
```

In each sub array, I want you to append to the 'performer' and 'song' the following 
1. append in the 3rd index of the array a brief explanation of which love song subtype category it fits best, or why it fits none
2. appending the 4th index of the array the love song subtype, or "none" if doesn't fit any of the love song subtypes

I want you to use these love song subtype definitions:
Serenade: A song directed at someone the singer loves, expressing their deep affection, undying love, infatuation, or praising someone as a soulmate.
- "Your Song" by Elton John
- "I Will Always Love You" by Whitney Houston
- "All of Me" by John Legend

Sexual Conquest: Songs that are less about the object of affection and more about the singer's own prowess or success in romantic pursuits.
- "I Get Around" by The Beach Boys
- "Blurred Lines" by Robin Thicke ft. Pharrell & T.I.
- "Womanizer" by Britney Spears
- "6ix9ine" by FEFE ft. Nicki Minaj, Murda Beatz

Love Songs for the Self: Songs that express self-love and affirmation, and standing up for oneself, often following a period of growth or learning from past relationships.
- "Thank U, Next" by Ariana Grande
- “Good As Hell” by Lizzo
- "Beautiful" by Christina Aguilera
- "Fighter" by Christina Aguilera

Unrequited: You've never had the person's love, you've never dated them, so you're admiring them from afar, often secretly, and fear you might not ever be loved by them.
- "You Belong With Me" by Taylor Swift
- "Creep" by Radiohead
- "Teardrops on My Guitar" by Taylor Swift
- "She's A Fool" by Lesley Gore
- "Jessie's Girl" by Rick Springfield

Good Riddance: songs about having had someone's love, but being glad now you've moved on (or realizing you should), often involving self-empowerment, anger, revenge, relief, or bitterness, often because of betrayal.
- "Since U Been Gone" by Kelly Clarkson
- "good 4 u" by Olivia Rodrigo
- "Tainted Love" by Soft Cell
- "Hit 'Em Up Style (Oops!)" by Blu Cantrell
- "Grenade" by Bruno Mars

Heartbreak & Longing: songs where you had someone's love but now it’s gone (either temporarily or forever) and you’re sad or regretful
- "Someone Like You" by Adele
- "Without You" by Mariah Carey
- "Jolene" by Dolly Parton
- "Ain't No Sunshine" by Bill Withers
- "Secret Love Song" by Little Mix and Jason Derulo
- "Miss You" by The Rolling Stones

It's Complicated: Songs about complicated relationships, often with conflicting emotions like love, hate, frustration, and desire intermingling, whether light-hearted or profound in tone.
- "Love Is a Battlefield" by Pat Benatar
- "Hot N Cold" by Katy Perry
- "Ex-Factor" by Lauryn Hill
- "Mood" by 24kGoldn

Platonic Love: Songs about non-romantic yet strong and meaningful relationships, such as friendships or familial bonds.
- "You're My Best Friend" by Queen
- “Dear Mama” by 2Pac
- "Wind Beneath My Wings" by Bette Midler

Here's an example correct response given the input above (with columns: 'performer', 'song', 'justification', 'love song subtype label'):

```
[
    ['twenty one pilots', 'Heathens', 'about social outcasts, not romance', 'none'],
    ['Little Mix and Jason Derulo', 'Secret Love Song', 'speakers long to be united with their lover, but must remain secret', 'Heartbreak & Longing'],
    ['will.i.am & Britney Spears', 'Scream & Shout', 'about dancing in the club, not romance', 'none'],
    ["21 Savage & Metro Boomin", "Runnin", 'a gangster rap, not romance', 'none'],
    ["6ix9ine Featuring Nicki Minaj & Murda Beatz", "FEFE", 'about sexual prowess with many sexual references', 'Sexual Conquest'],
    ['2Pac', "Dear Mama/Old School", 'love and appreciation for his mother', 'Platonic Love'],
    ["'N Sync", 'This I Promise You', 'promising to always love someone', 'Serenade'],
    ["Katy Perry", "Hot N Cold", 'about navigating a lover who cannot make up their mind and is inconsistent', "It's Complicated"],
    ["112", "Peaches & Cream", 'about sex & being addicted to a tasty woman', 'Sexual Conquest'],
    ["10cc", "I'm Not In Love", 'the speaker claims he does not love her, but the subtext is that he yearns to be with her', 'Heartbreak & Longing'],
    ["Olivia Rodrigo", "good 4 u", 'angry at how quickly their ex moved on', 'Good Riddance'],
]
```

It is crucial that you respond ONLY with the correct array in valid JSON format, do not include anything else in your response. In other words, your responce should be able to compiled to valid Python, do not include extraneous text outside the responce JSON array.
"""

### ... we tried without justification, results were not as good.

In [5]:
# NO justification asked for:
_NOT_USED__LOVE_SONG_LABELING_PROMPT = """
I will to give you a JSON list where each item is an array that contains an array where the first item is the performer and the second is the name of one of their songs from the Billboard Top 10 from 1958-2021.

Here's an example input, with 2 columns ('performer', 'song'):

```
[
    ['twenty one pilots', 'Heathens'],
    ['Little Mix and Jason Derulo'],
    ['will.i.am & Britney Spears', 'Scream & Shout'],
    ["21 Savage & Metro Boomin", "Runnin"],
    ["6ix9ine Featuring Nicki Minaj & Murda Beatz", "FEFE"],
    ['2Pac', "Dear Mama/Old School"],
    ["'N Sync", 'This I Promise You'],
    ["Katy Perry", "Hot N Cold"],
    ["112", "Peaches & Cream"],
    ["10cc", "I'm Not In Love"],
    ["Olivia Rodrigo", "good 4 u"],
]
```

In each sub array, I want you to append to the 'performer' and 'song' the following in the 3rd index of the array:
the love song subtype, or "none" if doesn't fit any of the love song subtypes.

I want you to use these love song subtype definitions:

Serenade: A song directed at someone the singer loves, expressing their deep affection, undying love, infatuation, or praising someone as a soulmate.
- "Your Song" by Elton John
- "I Will Always Love You" by Whitney Houston
- "All of Me" by John Legend

Sexual Conquest: Songs that are less about the object of affection and more about the singer's own prowess or success in romantic pursuits.
- "I Get Around" by The Beach Boys
- "Blurred Lines" by Robin Thicke ft. Pharrell & T.I.
- "Womanizer" by Britney Spears
- "6ix9ine" by FEFE ft. Nicki Minaj, Murda Beatz

Love Songs for the Self: Songs that express self-love and affirmation, and standing up for oneself, often following a period of growth or learning from past relationships.
- "Thank U, Next" by Ariana Grande
- “Good As Hell” by Lizzo
- "Beautiful" by Christina Aguilera
- "Fighter" by Christina Aguilera

Unrequited: You've never had the person's love, you've never dated them, so you're admiring them from afar, often secretly, and fear you might not ever be loved by them.
- "You Belong With Me" by Taylor Swift
- "Creep" by Radiohead
- "Teardrops on My Guitar" by Taylor Swift
- "She's A Fool" by Lesley Gore
- "Jessie's Girl" by Rick Springfield

Good Riddance: songs about having had someone's love, but being glad now you've moved on (or realizing you should), often involving self-empowerment, anger, revenge, relief, or bitterness, often because of betrayal.
- "Since U Been Gone" by Kelly Clarkson
- "good 4 u" by Olivia Rodrigo
- "Tainted Love" by Soft Cell
- "Hit 'Em Up Style (Oops!)" by Blu Cantrell
- "Grenade" by Bruno Mars

Heartbreak & Longing: songs where you had someone's love but now it’s gone (either temporarily or forever) and you’re sad or regretful
- "Someone Like You" by Adele
- "Without You" by Mariah Carey
- "Jolene" by Dolly Parton
- "Ain't No Sunshine" by Bill Withers
- "Secret Love Song" by Little Mix and Jason Derulo
- "Miss You" by The Rolling Stones

It's Complicated: Songs about complicated relationships, often with conflicting emotions like love, hate, frustration, and desire intermingling, whether light-hearted or profound in tone.
- "Love Is a Battlefield" by Pat Benatar
- "Hot N Cold" by Katy Perry
- "Ex-Factor" by Lauryn Hill
- "Mood" by 24kGoldn

Platonic Love: Songs about non-romantic yet strong and meaningful relationships, such as friendships or familial bonds.
- "You're My Best Friend" by Queen
- “Dear Mama” by 2Pac
- "Wind Beneath My Wings" by Bette Midler

Here's an example correct response given the input above (with columns: 'performer', 'song', 'justification', 'love song subtype label'):

```
[
    ['twenty one pilots', 'Heathens','none'],
    ['Little Mix and Jason Derulo', 'Secret Love Song', 'Heartbreak & Longing'],
    ['will.i.am & Britney Spears', 'Scream & Shout', 'none'],
    ["21 Savage & Metro Boomin", "Runnin", 'none'],
    ["6ix9ine Featuring Nicki Minaj & Murda Beatz", "FEFE", 'Sexual Conquest'],
    ['2Pac', "Dear Mama/Old School", 'Platonic Love'],
    ["'N Sync", 'This I Promise You', 'Serenade'],
    ["Katy Perry", "Hot N Cold","It's Complicated"],
    ["112", "Peaches & Cream", 'Sexual Conquest'],
    ["10cc", "I'm Not In Love", 'Heartbreak & Longing'],
    ["Olivia Rodrigo", "good 4 u", 'Good Riddance'],
]
```

It is crucial that you respond ONLY with the correct array in valid JSON format, do not include anything else in your response. In other words, your responce should be able to compiled to valid Python, do not include extraneous text outside the responce JSON array.
"""

In [6]:
import ast
ast.literal_eval("['Michael Jackson']")
def eval_this_str(s):
    return ast.literal_eval(s)

In [7]:
def get_love_song_labels_for_rows(row_subset_str):
    try: 
        completion = openai.ChatCompletion.create(
#           model="gpt-3.5-turbo",
          model="gpt-4",
          messages=[
            {"role": "user", "content": LOVE_SONG_LABELING_PROMPT},
            {"role": "user", "content": row_subset_str},
          ]
        )
        return eval_this_str(completion.choices[0].message['content'])
    except Exception as e:
        print('ERROR FETCHING', e)
        return []

# res = get_love_song_labels_for_rows(row_subset_str=TEST_SUBSET_FROM_DF)

### Setup API call batching code

In [8]:
INCREMENT = 50

In [9]:
grouped_by_song_df = gender_df.groupby(['performer', 'song']).first().reset_index()

In [10]:
def fetch_all_data_in_batches(df, increment=INCREMENT):
    gpt_labeled_result = []
    for i in range(101):
        subset = df[['performer', 'song']][i*increment : (i+1)*increment]

        if len(subset):
            subset_as_json_string = subset.to_json(orient='values')
            print([i*increment, (i+1)*increment])
            print(subset_as_json_string, '\n')
            fetched_and_formatted_res = get_love_song_labels_for_rows(subset_as_json_string)
            print(fetched_and_formatted_res)
            gpt_labeled_result = [*gpt_labeled_result, *fetched_and_formatted_res]
    return gpt_labeled_result

In [11]:
def save_to_temp_csv(gpt_labeled_result, run_num):
    first_label_pass_df = pd.DataFrame(
        gpt_labeled_result, 
        columns=['performer', 'song', 
                 'justification',
                 'love_song_sub_type']
    )
    first_label_pass_df.to_csv(f'./data/15-TEMP-gpt4-sub-types-{run_num}.csv', index=False)

# Execute calls (in batched batches in case of failure)

### For50 songs :
- With justification -- GP3: 40 seconds, GPT4: 4 min 
- without -- GPT3: ..., GPT4: 2 min, 1:30 min

However, we see HUGE accuracy boost both from:
a. having it give a justification before the label
b. using GPT4, not 3

In [12]:
songs_within_gpt_dataset_df = grouped_by_song_df[
    grouped_by_song_df.date_as_decimal < 2021.5
]

In [21]:
# %%time
# save_to_temp_csv(
#     fetch_all_data_in_batches(
#         songs_within_gpt_dataset_df.iloc[0:1000]
#     ), 
#     1
# )
# etc for 2-5

## Refetch the failed calls:

In [22]:
first_round_fetch_df = pd.concat([
    pd.read_csv(f'./data/15-TEMP-gpt4-sub-types-{1}.csv'),
    pd.read_csv(f'./data/15-TEMP-gpt4-sub-types-{2}.csv'),
    pd.read_csv(f'./data/15-TEMP-gpt4-sub-types-{3}.csv'),
    pd.read_csv(f'./data/15-TEMP-gpt4-sub-types-{4}.csv'),
    pd.read_csv(f'./data/15-TEMP-gpt4-sub-types-{5}.csv'),
])

In [27]:
result_df = songs_within_gpt_dataset_df.merge(first_round_fetch_df, on=['performer', 'song'], how='outer', indicator=True)
rows_not_in_first_round = result_df[result_df['_merge'] == 'left_only'].drop(columns=['_merge'])
# rows_not_in_first_round

In [33]:
# %%time
# save_to_temp_csv(
#     fetch_all_data_in_batches(
#         rows_not_in_first_round
#     ), 
#     'redo-the-failed-calls-1'
# )

# Stitch it all together for a quick & dirty final version (it's ok if we skipped some songs)

In [44]:
love_song_subtypes_df = pd.concat([
    pd.read_csv(f'./data/15-TEMP-gpt4-sub-types-{1}.csv'),
    pd.read_csv(f'./data/15-TEMP-gpt4-sub-types-{2}.csv'),
    pd.read_csv(f'./data/15-TEMP-gpt4-sub-types-{3}.csv'),
    pd.read_csv(f'./data/15-TEMP-gpt4-sub-types-{4}.csv'),
    pd.read_csv(f'./data/15-TEMP-gpt4-sub-types-{5}.csv'),
    pd.read_csv(f'./data/15-TEMP-gpt4-sub-types-{"redo-the-failed-calls-1"}.csv'),
])
love_song_subtypes_df


# replace "None" with "none"
love_song_subtypes_df.love_song_sub_type.replace({'None': 'none'}, inplace=True)
# only keep rows that match our original 9 categories (ie the top most popular ones)
LOVE_SONG_SUB_TYPES = [
    'none', 'Serenade', 'Heartbreak & Longing', 'Sexual Conquest',
       "It's Complicated", 'Good Riddance', 'Unrequited',
       'Love Songs for the Self', 'Platonic Love',
]
love_song_subtypes_df = love_song_subtypes_df[
    love_song_subtypes_df.love_song_sub_type.apply(lambda val: val in LOVE_SONG_SUB_TYPES)
]

love_song_subtypes_df.to_csv('./data/15-OUTPUT-gpt4-love-song-subtypes-rough-draft.csv', index=False)

love_song_subtypes_df.love_song_sub_type.value_counts()

none                       1716
Serenade                   1044
Heartbreak & Longing        702
Sexual Conquest             413
It's Complicated            399
Good Riddance               211
Unrequited                  203
Love Songs for the Self      95
Platonic Love                81
Name: love_song_sub_type, dtype: int64